# Here is a title
here is some text

In [1]:
#%%capture 
#hide output of this cell

import tensorflow as tf
print(tf.__version__)


# Qua fatti una seconda var con il tuo path, cosi a seconda di chi lo sta usando mettiamo un hastag allinizio
tfrecord_path = 'C:/Users/matthias/Documents/projects/audioset_v1_embeddings/bal_train/--.tfrecord'


2.10.0


First of all, inspect the raw TFR file structure to get an idea of how its built and it looks. 

In [4]:
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Iterate through the dataset and print the raw example data
for record in dataset:
    example = tf.train.Example.FromString(record.numpy())
    print(example)


features {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 40.0
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 399
      }
    }
  }
  feature {
    key: "start_time_seconds"
    value {
      float_list {
        value: 30.0
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "--cB2ZVjpnA"
      }
    }
  }
}

features {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 40.0
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 0
        value: 451
      }
    }
  }
  feature {
    key: "start_time_seconds"
    value {
      float_list {
        value: 30.0
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "--PJHxphWEs"
      }
    }
  }
}

features {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 40

Now iterate trough all files of the google AudioSet dataset. Using `filter_for_thunder()` we isolate all entries that have the thunder label and create a new file with them. 

In [5]:
# Create a TFRecord writer for the new file
def filter_for_thunder(old_file_path, new_file_path, value_to_check, entries_list):
    """Parse trough a tfrecord file and search for entries related to one specific label, for example thunder (287),
    when found, append the whole entrie to a new file.

    Args:
        old_file_path (str): path of the original TFRecord file
        new_file_path (str): path to the newly created file containing only the selected entries
        value_to_check (int): value of the label to use as filter 
        entries_list (empty list): an empty list to append matching entries to 
    """    
    with tf.io.TFRecordWriter(new_file_path) as writer: 
        
        for raw_record in tf.data.TFRecordDataset(old_file_path):
            example = tf.train.Example()
            example.ParseFromString(raw_record.numpy())

            if 'labels' in example.features.feature:
                label_values = example.features.feature['labels'].int64_list.value
                if value_to_check in label_values:
                    # If the value is found, write the example to the new TFRecord file
                    writer.write(raw_record.numpy())

In [ ]:
#TODO: IMPLEMENT THIS IN THE CODE ABOVE, as it is not possible to append stuff to TFR files, we need to append 
# all mathing entries to an empty list and write the file once the loop is done 



# Define the path to the original TFRecord file and the new TFRecord file
original_filename = 'original.tfrecord'
new_filename = 'new_filtered.tfrecord'

# Define the value to check for (e.g., 287)
value_to_check = 287

# Create a list to store the matching entries
matching_entries = []

# Read the original TFRecord file and append the matching entries to the list
for raw_record in tf.data.TFRecordDataset(original_filename):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())

    if 'labels' in example.features.feature:
        label_values = example.features.feature['labels'].int64_list.value
        if value_to_check in label_values:
            matching_entries.append(raw_record.numpy())

# Write the updated list to the new TFRecord file
with tf.io.TFRecordWriter(new_filename) as writer:
    for entry in matching_entries:
        writer.write(entry)


In [6]:
#Use the function on a single file to check functionality, can probably be deleted later on
new_filename = 'thunder_tryout.tfrecord'
# Define the value to check for (e.g., 287)
value_to_check = 95

filter_for_thunder(tfrecord_path, new_filename, value_to_check)



ValueError: Not a valid compression_type: "a"

In [71]:
# check the newly created file if it containes the entries we expect
dataset2 = tf.data.TFRecordDataset(new_filename)

# Iterate through the dataset and print the raw example data
for record in dataset2:
    example = tf.train.Example.FromString(record.numpy())

features {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 10.0
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 0
        value: 95
        value: 137
      }
    }
  }
  feature {
    key: "start_time_seconds"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "--aE2O5G5WE"
      }
    }
  }
}
 1


# Junk code, keep for now, delete later

In [57]:
# Nuovo approcio da stack overflow: https://stackoverflow.com/questions/65007191/how-to-read-decode-tfrecords-file-see-the-images-inside-and-do-augmentation
# questo funyiona un pó meglio, si riescono ad accedere le singole entry. Sembra che alla fine file 
# TFRecord siano dizionari di dizionari di dizionari o qualcosa del genere. 
dataset = tf.data.TFRecordDataset(tfrecord_path)


val = 1
info = {}


for i, raw_record in enumerate(dataset): # keeping at a minimal example of 3, remove for actual code
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    
    # Questa parte qua fa il check per vedere se trova entries con thunder (287)
    if 'labels' in example.features.feature:
        label_values = example.features.feature['labels'].int64_list.value
        if 399 in label_values:
            print('THUNDEEEEERRRRR')
            print(example)
    
    
"""     for k, v in example.features.feature.items():
        if k == 'labels':
            #print(k, v, 'NEXT')
            info[k] = v
            
            val += 1 """
            
            
            

""" print('INFO: \n', info) """
#for n, m in info.items():
 #   if 287 in m:
  #      print('THUNDEERRRRRR')




            

THUNDEEEEERRRRR
features {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 40.0
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 399
      }
    }
  }
  feature {
    key: "start_time_seconds"
    value {
      float_list {
        value: 30.0
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "--cB2ZVjpnA"
      }
    }
  }
}



" print('INFO: \n', info) "

In [4]:
# Qua faccio un check per capire il data type del feature labels, perché piú avanti mi crea problemi
# Define a parsing function to extract and decode the data
## Boh sto qua semplicemente restituisce il data type specificato su tf.io.VarLenFeature(tf.int64), 
## e non il data type reale. Anche cambiare la parte 'features/feature/video_id' non ha nessun effetto


def parse_function(example_proto):
    feature_description = {
        'features/feature/video_id': tf.io.VarLenFeature(tf.int64),
        # Add other features and their types based on your TFRecord structure
    }

    # Parse the example
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # Get the 'labels' feature
    labels = example['features/feature/video_id']
    
    return labels

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Map the parsing function to the dataset
parsed_data = dataset.map(parse_function)

# Print the data type of the 'labels' feature
for labels in parsed_data:
    dtype = labels.values.dtype
    print(f'Data Type of "labels" Feature: {dtype}')

Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>
Data Type of "labels" Feature: <dtype: 'int64'>


Open and explore TFRecord files. 
Goal is to filter them in order to only keep files containing a certain label, like thunder

In [20]:
#Questo approcio é quello che mi da chatGTP, che peró per me non funziona, le entry che mi ritorna sono vuote. 
# Qua poi in teoria volgio leggere i label (e solo quelli) in modo che piú avanti si possano filtrare i file che contengono il label thunder (287,/m/0ngt1)
# La lista dei vari label si puó trovare nel file csv che ho caricato

# Define a parsing function to extract and decode the data
def parse_function(example_proto):
    feature_description = {
        'features/feature/': tf.io.VarLenFeature(tf.float32),
        # Add other features and their types based on your TFRecord structure
    }

    # Parse the example
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # Decode the 'labels' feature
    labels = tf.sparse.to_dense(example['features/feature/'])
    
    return labels

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Map the parsing function to the dataset and iterate through the records
for labels in dataset.map(parse_function):
    # Print the 'labels' feature
    print(f'time: {labels.numpy()}')

time: []
time: []
time: []
time: []
time: []
time: []
time: []


In [11]:
tfrecord_path = 'C:/Users/matthias/Documents/projects/audioset_v1_embeddings/bal_train/-0.tfrecord'

# tfrecord_dataset = tf.data.TFRecordDataset(tfrecord_path)


# Define a parsing function to extract and decode the data
def parse_function(example_proto):
    feature_description = {
        'context/feature/video_id': tf.io.FixedLenFeature([], tf.string),
        'context/feature/start_time_seconds': tf.io.FixedLenFeature([], tf.float32),
        'context/feature/end_time_seconds': tf.io.FixedLenFeature([], tf.float32),
        'context/feature/labels': tf.io.VarLenFeature(tf.int64),
        'feature_lists/feature_list/audio_embedding': tf.io.VarLenFeature(tf.string),
        # Define other features and their types based on your TFRecord structure
    }
    
    # Parse the example
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # If 'end_time_seconds' may not be present in all records, use allow_missing=True
    if 'context/feature/end_time_seconds' in example:
        end_time_seconds = example['context/feature/end_time_seconds']
    else:
        end_time_seconds = 0.0  # Provide a default value
    
    
    # Decode the VarLenFeatures
    example['context/feature/labels'] = tf.sparse.to_dense(example['context/feature/labels'], default_value=0)
    example['feature_lists/feature_list/audio_embedding'] = tf.sparse.to_dense(example['feature_lists/feature_list/audio_embedding'], default_value='b')
    
    return example

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_path)

# Map the parsing function to the dataset and iterate through the records
for record in dataset.map(parse_function):
    video_id = tf.strings.decode(record['context/feature/video_id'], 'utf-8')
    start_time_seconds = tf.make_ndarray(record['context/feature/start_time_seconds'])[0]
    end_time_seconds = tf.make_ndarray(record['context/feature/end_time_seconds'])[0]
    labels = tf.make_ndarray(record['context/feature/labels'])
    audio_embedding = record['feature_lists/feature_list/audio_embedding']
    
    print(f'Video ID: {video_id}')
    print(f'Start Time (seconds): {start_time_seconds}')
    print(f'End Time (seconds): {end_time_seconds}')
    print(f'Labels: {labels}')
    print(f'Audio Embedding: {audio_embedding}')



InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Feature: context/feature/end_time_seconds (data type: float) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext]

In [ ]:
# 